In [ ]:
# from typing import Literal
# import torch
# from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig
# from pathlib import Path


# class SmolVLM2ChatModel:
#     """
#     LangChain-style wrapper for HuggingFace SmolVLM2 that supports chat-like .invoke() calls.
#     Automatically selects between 256M, 500M, and 2.2B model variants based on GPU memory.
#     Supports quantization using bitsandbytes (4-bit or 8-bit).
#     """

#     MODEL_MAP = {
#         "small": "HuggingFaceTB/SmolVLM2-256M-Instruct",
#         "medium": "HuggingFaceTB/SmolVLM2-500M-Instruct",
#         "large": "HuggingFaceTB/SmolVLM2-2.2B-Instruct",
#     }

#     def __init__(
#         self,
#         model_size: Literal["small", "medium", "large"] | None = None,
#         device: str | None = None,
#         dtype: torch.dtype = torch.bfloat16,
#         quantization: Literal["none", "8bit", "4bit"] = "none",
#         model_root: str | Path | None = None,  # 👈 optionally specify local model root
#     ):
#         self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
#         self.quantization = quantization

#         # === 1. Auto-select model size if not given ===
#         if model_size is None:
#             model_size = self._auto_select_model_size()

#         self.model_name = self.MODEL_MAP.get(model_size, self.MODEL_MAP["medium"])
#         print(f"🔹 Preparing to load {self.model_name} ({model_size}) with {quantization} quantization...")

#         # === 2. Define local cache directory ===
#         model_root = Path(model_root) if model_root else Path(__file__).parent.parent / "model"
#         self.model_dir = model_root / self.model_name.split("/")[-1]
#         self.model_dir.mkdir(parents=True, exist_ok=True)
#         print(f"📂 Model cache directory: {self.model_dir}")

#         # === 3. Configure quantization ===
#         quant_config = None
#         if quantization in ["8bit", "4bit"]:
#             try:
#                 quant_config = BitsAndBytesConfig(
#                     load_in_8bit=(quantization == "8bit"),
#                     load_in_4bit=(quantization == "4bit"),
#                     bnb_4bit_quant_type="nf4",
#                     bnb_4bit_use_double_quant=True,
#                     bnb_4bit_compute_dtype=torch.float16,
#                 )
#                 print(f"⚙️ Using bitsandbytes {quantization} quantization.")
#             except Exception as e:
#                 print("⚠️ Quantization unavailable, proceeding without it:", e)

#         # === 4. Load processor ===
#         self.processor = AutoProcessor.from_pretrained(
#             self.model_name,
#             cache_dir=self.model_dir
#         )

#         # === 5. Load model ===
#         print(f"🚀 Loading model weights into {self.device}...")
#         self.model = AutoModelForImageTextToText.from_pretrained(
#             self.model_name,
#             cache_dir=self.model_dir,
#             torch_dtype=dtype,
#             quantization_config=quant_config,
#             device_map="auto" if quant_config else None,
#         )

#         # === 6. Move to device if not quantized ===
#         if not quant_config:
#             self.model.to(self.device)

#         print(f"✅ Successfully loaded {model_size.upper()} SmolVLM2 model ({self.device})!")

#     # -------------------------------------------------------------------------
#     def _auto_select_model_size(self):
#         """Automatically select model size based on available GPU VRAM."""
#         if not torch.cuda.is_available():
#             print("⚠️ CUDA not available. Using CPU-friendly 256M model.")
#             return "small"

#         try:
#             total_vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
#             print(f"🧮 Detected VRAM: {total_vram_gb:.1f} GB")

#             if total_vram_gb < 6:
#                 return "small"
#             elif total_vram_gb < 10:
#                 return "medium"
#             else:
#                 return "large"
#         except Exception as e:
#             print("⚠️ Could not detect VRAM:", e)
#             return "medium"

#     # -------------------------------------------------------------------------
#     def _format_messages(self, messages):
#         """Format messages for processor chat template."""
#         if not isinstance(messages, list):
#             raise ValueError("messages must be a list of message dicts.")
#         return messages

#     # -------------------------------------------------------------------------
#     def _generate(self, formatted_messages, max_new_tokens=512, do_sample=False):
#         inputs = self.processor.apply_chat_template(
#             formatted_messages,
#             add_generation_prompt=True,
#             tokenize=True,
#             return_dict=True,
#             return_tensors="pt",
#         ).to(self.device, dtype=torch.bfloat16)

#         outputs = self.model.generate(
#             **inputs,
#             do_sample=do_sample,
#             max_new_tokens=max_new_tokens,
#         )

#         return self.processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()

#     # -------------------------------------------------------------------------
#     def invoke(self, input):
#         """LangChain-style .invoke() method for single chat turn."""
#         formatted_messages = self._format_messages([input])
#         return {"type": "ai_message", "content": self._generate(formatted_messages)}

#     # -------------------------------------------------------------------------
#     def batch(self, inputs):
#         """LangChain-style .batch() method for multiple chat turns."""
#         results = []
#         for inp in inputs:
#             formatted_messages = self._format_messages([inp])
#             response = self._generate(formatted_messages)
#             results.append({"type": "ai_message", "content": response})
#         return results


In [1]:
from video_agent import SmolVLM2ChatModel
from tools import extract_assistant_response

chat_model = SmolVLM2ChatModel(quantization="4bit", model_size="medium")



🔹 Preparing to load HuggingFaceTB/SmolVLM2-500M-Instruct (medium) with 4bit quantization...
📂 Model cache directory: c:\Interview Assignment\Intel GenAI Software Engineer Assessment\Short-Video-Analyst\model\SmolVLM2-500M-Instruct
⚙️ Using bitsandbytes 4bit quantization.


processor_config.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

c:\Users\jthxc\anaconda3\envs\jaredllm\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Interview Assignment\Intel GenAI Software Engineer Assessment\Short-Video-Analyst\model\SmolVLM2-500M-Instruct\models--HuggingFaceTB--SmolVLM2-500M-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


chat_template.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

✅ Successfully loaded MEDIUM SmolVLM2 model (cuda)!


In [ ]:
# import re

# def extract_assistant_response(text: str) -> str:
#     """
#     Extracts the Assistant's actual response text from a raw SmolVLM output.

#     Example:
#         Input:
#             "🧠 Response: User: You are provided the following... Assistant: Yes, there is a graph..."
#         Output:
#             "Yes, there is a graph in the video. It shows the progress..."
#     """
#     # Remove leading emojis or response prefixes like "🧠 Response:"
#     cleaned_text = re.sub(r"^🧠\s*Response:\s*", "", text.strip())

#     # Use regex to find text after 'Assistant:'
#     match = re.search(r"Assistant:\s*(.*)", cleaned_text, re.DOTALL)
#     if match:
#         # Clean trailing whitespace/newlines
#         return match.group(1).strip()
#     return cleaned_text.strip()


In [2]:
# Example 1: single invoke
#video_path = r"C:\Interview Assignment\Intel GenAI Software Engineer Assessment\Short-Video-Analyst\sample_videos\What is MCP and how does it change AI_ (MCP explained) #ai #artificialintelligence (1).mp4"

#video_path = r"C:\Interview Assignment\Intel GenAI Software Engineer Assessment\Short-Video-Analyst\sample_videos\cat.mp4"

video_path = r"C:\Interview Assignment\Intel GenAI Software Engineer Assessment\Short-Video-Analyst\sample_videos\Son strikes first 🚀.mp4"

#video_path = r"C:\Interview Assignment\Intel GenAI Software Engineer Assessment\Short-Video-Analyst\sample_videos\Robbing a bank in 2028.mp4"

#video_path = r"C:\Interview Assignment\Intel GenAI Software Engineer Assessment\Short-Video-Analyst\sample_videos\n8n autoamtion.mp4"

user_query = "Describe what you see in the video"

response = chat_model.invoke({
    "role": "user",
    "content": [
        {"type": "video", "path": video_path},
        {"type": "text", "text": user_query}
    ]
})

ai_msg = extract_assistant_response(response["content"])

print(f"{ai_msg}")


c:\Users\jthxc\anaconda3\envs\jaredllm\Lib\site-packages\transformers\video_processing_utils.py:879: UserWarning: `torchcodec` is not installed and cannot be used to decode the video by default. Falling back to `torchvision`. Note that `torchvision` decoding is deprecated and will be removed in future versions. 
  warnings.warn(
c:\Users\jthxc\anaconda3\envs\jaredllm\Lib\site-packages\transformers\video_utils.py:524: UserWarning: Using `torchvision` for video decoding is deprecated and will be removed in future versions. Please use `torchcodec` instead.
  warnings.warn(
You have used fast image processor with LANCZOS resample which not yet supported for torch.Tensor. BICUBIC resample will be used as an alternative. Please fall back to image processor if you want full consistency with the original model.
c:\Users\jthxc\anaconda3\envs\jaredllm\Lib\site-packages\transformers\integrations\sdpa_attention.py:96: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally

The video captures a soccer match between FC Barcelona and Atletico Madrid, featuring a player in a white and gold uniform celebrating a goal. The scene is set in a stadium filled with spectators, with a Coca-Cola advertisement visible in the background. The player in the white and gold uniform is seen celebrating the goal, with his arms raised in the air, and his teammates nearby. The video then shifts to a different scene, showing a player in a white and gold uniform celebrating a goal, with a crowd of spectators in the background. The scene then transitions to a different player in a white and gold uniform, who is seen celebrating a goal, with a crowd of spectators in the background. The video continues with a player in a white and gold uniform celebrating a goal, with a crowd of spectators in the background. The scene then shifts to a player in a white and gold uniform celebrating a goal, with a crowd of spectators in the background. The video continues with a player in a white and

In [6]:
response["content"]

"User: You are provided the following series of twenty-three frames from a 0:00:23 [H:MM:SS] video.\n\nFrame from 00:00:\nFrame from 00:01:\nFrame from 00:02:\nFrame from 00:03:\nFrame from 00:04:\nFrame from 00:05:\nFrame from 00:06:\nFrame from 00:07:\nFrame from 00:08:\nFrame from 00:09:\nFrame from 00:10:\nFrame from 00:11:\nFrame from 00:12:\nFrame from 00:13:\nFrame from 00:14:\nFrame from 00:15:\nFrame from 00:16:\nFrame from 00:18:\nFrame from 00:19:\nFrame from 00:20:\nFrame from 00:21:\nFrame from 00:22:\nFrame from 00:23:\n\nCan you tell me the objects that are present in the video ?\nAssistant: The video features a white and brown cat with a red bandana around its neck. The cat is sitting on a tiled floor, which is part of a kitchen area. The cat is positioned in the center of the frame, and it appears to be in a playful or curious mood. The cat is looking towards the camera, which suggests that it is interested in what the camera is capturing. The cat's position and the an

# 2. Version 2

In [1]:
from typing import Literal
import torch
import cv2
import numpy as np
from pathlib import Path
from PIL import Image
from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig
import re

class SmolVLM2ChatModel:
    """
    LangChain-style wrapper for HuggingFace SmolVLM2 that supports:
      ✅ chat-like .invoke() calls
      ✅ automatic model size selection
      ✅ quantization (4-bit / 8-bit)
      ✅ video analysis of up to ~1 minute (frame sampling + summarization)
    """

    MODEL_MAP = {
        "small": "HuggingFaceTB/SmolVLM2-256M-Instruct",
        "medium": "HuggingFaceTB/SmolVLM2-500M-Instruct",
        "large": "HuggingFaceTB/SmolVLM2-2.2B-Instruct",
    }

    def __init__(
        self,
        model_size: Literal["small", "medium", "large"] | None = None,
        device: str | None = None,
        dtype: torch.dtype = torch.bfloat16,
        quantization: Literal["none", "8bit", "4bit"] = "none",
    ):
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.quantization = quantization

        # Auto-select model size if not specified
        if model_size is None:
            model_size = self._auto_select_model_size()

        self.model_name = self.MODEL_MAP.get(model_size, self.MODEL_MAP["medium"])
        print(f"🔹 Loading {self.model_name} on {self.device} with {quantization} quantization...")

        # Configure quantization if requested
        quant_config = None
        if quantization in ["8bit", "4bit"]:
            try:
                quant_config = BitsAndBytesConfig(
                    load_in_8bit=(quantization == "8bit"),
                    load_in_4bit=(quantization == "4bit"),
                    bnb_4bit_quant_type="nf4",
                    bnb_4bit_use_double_quant=True,
                    bnb_4bit_compute_dtype=dtype,
                )
                print(f"⚙️ Using {quantization} quantization with bitsandbytes.")
            except Exception as e:
                print("⚠️ Quantization not available, loading normally:", e)

        # Load processor and model
        self.processor = AutoProcessor.from_pretrained(self.model_name)
        self.model = AutoModelForImageTextToText.from_pretrained(
            self.model_name,
            torch_dtype=dtype,
            device_map="auto" if quant_config else None,
            quantization_config=quant_config,
        ).to(self.device if not quant_config else None)

        print(f"✅ {model_size.upper()} SmolVLM2 model loaded successfully!\n")

    # -------------------------------------------------------------------------
    def _auto_select_model_size(self):
        """Automatically select model size based on available GPU VRAM."""
        if not torch.cuda.is_available():
            print("⚠️ CUDA not available. Using CPU-friendly 256M model.")
            return "small"

        try:
            total_vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
            print(f"🧮 Detected VRAM: {total_vram_gb:.1f} GB")

            if total_vram_gb < 6:
                return "small"
            elif total_vram_gb < 10:
                return "medium"
            else:
                return "large"
        except Exception as e:
            print("⚠️ Could not detect VRAM:", e)
            return "medium"

    # -------------------------------------------------------------------------
    def _format_messages(self, messages):
        """Format messages for chat template."""
        if not isinstance(messages, list):
            raise ValueError("messages must be a list of message dicts.")
        return messages

    # -------------------------------------------------------------------------
    def _generate(self, formatted_messages, max_new_tokens=512, do_sample=False):
        inputs = self.processor.apply_chat_template(
            formatted_messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt",
        ).to(self.device, dtype=torch.bfloat16)

        outputs = self.model.generate(**inputs, do_sample=do_sample, max_new_tokens=max_new_tokens)
        return self.processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()

    # -------------------------------------------------------------------------
    def invoke(self, input):
        """LangChain-style .invoke() method for single chat turn."""
        formatted_messages = self._format_messages([input])
        return {"type": "ai_message", "content": self._generate(formatted_messages)}

    def batch(self, inputs):
        """LangChain-style .batch() method for multiple chat turns."""
        results = []
        for inp in inputs:
            formatted_messages = self._format_messages([inp])
            response = self._generate(formatted_messages)
            results.append({"type": "ai_message", "content": response})
        return results

    # -------------------------------------------------------------------------
    # 🎥 --- Video Analysis Extension ---
    # -------------------------------------------------------------------------

    def _sample_video_frames(self, video_path: str, num_frames: int = 12):
        """Sample evenly spaced frames from the video."""
        cap = cv2.VideoCapture(str(video_path))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)

        output_dir = Path("sampled_frames")
        output_dir.mkdir(exist_ok=True)

        frame_paths = []
        for i in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if ret:
                frame_path = output_dir / f"frame_{i}.jpg"
                cv2.imwrite(str(frame_path), frame)
                frame_paths.append(str(frame_path))
        cap.release()
        return frame_paths

    def _analyze_frame(self, frame_path: str, question: str, idx: int, total: int):
        """Analyze a single frame using SmolVLM2."""
        image = Image.open(frame_path)
        # ✅ Add <image> token so SmolVLM2 knows an image is provided
        prompt = f"<image>\nFrame {idx+1}/{total}: {question}"
        inputs = self.processor(text=prompt, images=[image], return_tensors="pt").to(self.device)
        output = self.model.generate(**inputs, max_new_tokens=120)
        return self.processor.batch_decode(output, skip_special_tokens=True)[0]

 

    def analyze_video(self, video_path: str, question: str, num_frames: int = 12):
        """Analyze a video and return cleaned, non-empty frame results while preserving the user query."""
        print(f"🎬 Sampling {num_frames} frames from {video_path}...")
        frame_paths = self._sample_video_frames(video_path, num_frames)
        print(f"✅ {len(frame_paths)} frames extracted.\n")

        frame_results = []

        def clean_text(text: str) -> str:
            """Clean model output but keep the original user question."""
            text = text.strip()

            # Remove the "Frame X/Y:" prefix only
            text = re.sub(r"^Frame\s*\d+/\d+:\s*", "", text, flags=re.IGNORECASE)

            # Remove empty lines and redundant spaces
            text = re.sub(r"\s+", " ", text).strip()

            # Remove consecutive duplicate sentences
            sentences = re.split(r"(?<=[.!?])\s+", text)
            seen = set()
            cleaned_sentences = []
            for s in sentences:
                s_clean = s.strip()
                if s_clean and s_clean.lower() not in seen:
                    cleaned_sentences.append(s_clean)
                    seen.add(s_clean.lower())
            text = " ".join(cleaned_sentences)

            # If text only repeats the user query (no description), skip it later
            return text.strip()

        for idx, frame_path in enumerate(frame_paths):
            print(f"🖼️ Analyzing frame {idx+1}/{len(frame_paths)}...")
            try:
                result = self._analyze_frame(frame_path, question, idx, len(frame_paths))
                cleaned = clean_text(result)

                # Skip empty or echo-only results
                if cleaned and cleaned.lower() != question.lower():
                    print(f"📝 Frame {idx+1} cleaned result:", cleaned, "\n")
                    frame_results.append({
                        "frame_index": idx,
                        "frame_path": frame_path,
                        "analysis": cleaned
                    })
                else:
                    print(f"⚠️ Frame {idx+1} produced no new information, skipping.\n")

            except Exception as e:
                print(f"❌ Error analyzing frame {idx+1}: {e}")

        print(f"✅ Video analysis complete. {len(frame_results)} valid, cleaned analyses collected.")
        return {
            "frames_analyzed": len(frame_results),
            "results": frame_results
        }




In [2]:
model = SmolVLM2ChatModel(model_size="medium", quantization="4bit")

🔹 Loading HuggingFaceTB/SmolVLM2-500M-Instruct on cuda with 4bit quantization...
⚙️ Using 4bit quantization with bitsandbytes.


`torch_dtype` is deprecated! Use `dtype` instead!


✅ MEDIUM SmolVLM2 model loaded successfully!



In [3]:
video_path = r"C:\Interview Assignment\Intel GenAI Software Engineer Assessment\Short-Video-Analyst\sample_videos\What is MCP and how does it change AI_ (MCP explained) #ai #artificialintelligence.mp4"

video_path = r"C:\Interview Assignment\Intel GenAI Software Engineer Assessment\Short-Video-Analyst\sample_videos\cat.mp4"

result = model.analyze_video(
    video_path=video_path,
    question="Describe the main actions, objects, and sequence of events.",
    num_frames=16
)

print(result["results"])


🎬 Sampling 16 frames from C:\Interview Assignment\Intel GenAI Software Engineer Assessment\Short-Video-Analyst\sample_videos\cat.mp4...
✅ 16 frames extracted.

🖼️ Analyzing frame 1/16...


c:\Users\jthxc\anaconda3\envs\jaredllm\Lib\site-packages\transformers\integrations\sdpa_attention.py:96: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


📝 Frame 1 cleaned result: Describe the main actions, objects, and sequence of events. The cat is sitting on the floor, looking at the camera. The cat is wearing a red and white bandana. The cat is chewing on its paw. The cat is sitting on the floor. The cat is looking at the camera. 

🖼️ Analyzing frame 2/16...
📝 Frame 2 cleaned result: Describe the main actions, objects, and sequence of events. The cat is sitting on the floor, looking at the camera. The cat is wearing a red bandana. The cat is sitting on the floor. The cat is looking at the camera. The cat 

🖼️ Analyzing frame 3/16...
📝 Frame 3 cleaned result: Describe the main actions, objects, and sequence of events. The cat is sitting on the floor, looking at the camera. The cat is wearing a red collar. The background is a kitchen with a refrigerator. The cat is sitting on the floor. The cat is looking at the camera. The cat 

🖼️ Analyzing frame 4/16...
📝 Frame 4 cleaned result: Describe the main actions, objects, and sequence of e

In [4]:
result

{'frames_analyzed': 4,
 'results': [{'frame_index': 5,
   'frame_path': 'sampled_frames\\frame_791.jpg',
   'analysis': 'Describe the main actions, objects, and sequence of events. A diagram of a software development process is shown. The diagram shows a flowchart with the following steps: After MCP, LLM, MCP, and Model context. The diagram also includes icons for LLM, MCP, and Model context. The background is dark. The flowchart is divided into three main sections: LLM, MCP, and Model context. Each section has an icon and a text label. The LLM section has an icon of a laptop and a text label that reads "LSTM". The MCP section has an icon of a computer and'},
  {'frame_index': 8,
   'frame_path': 'sampled_frames\\frame_1266.jpg',
   'analysis': 'Describe the main actions, objects, and sequence of events. A person is speaking into a microphone in a recording studio. The background is a studio with a microphone and a computer monitor. The person is wearing a black shirt. The scene is set

# 3. Version 3

In [19]:
from typing import Literal
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig
from moviepy import VideoFileClip
from pathlib import Path


class SmolVLM2ChatModel:
    """
    Extended SmolVLM2 wrapper that:
      ✅ Auto-selects model size based on GPU VRAM
      ✅ Supports bitsandbytes 4/8-bit quantization
      ✅ Uses new MoviePy API (.subclipped(), .with_volume_scaled())
      ✅ Trims videos into 20-second clips and analyzes each one
    """

    MODEL_MAP = {
        "small": "HuggingFaceTB/SmolVLM2-256M-Instruct",
        "medium": "HuggingFaceTB/SmolVLM2-500M-Instruct",
        "large": "HuggingFaceTB/SmolVLM2-2.2B-Instruct",
    }

    def __init__(
        self,
        model_size: Literal["small", "medium", "large"] | None = None,
        device: str | None = None,
        dtype: torch.dtype = torch.bfloat16,
        quantization: Literal["none", "8bit", "4bit"] = "none",
    ):
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.quantization = quantization

        # 🔍 Auto-select model size based on VRAM if not specified
        if model_size is None:
            model_size = self._auto_select_model_size()

        self.model_name = self.MODEL_MAP.get(model_size, self.MODEL_MAP["medium"])
        print(f"🔹 Loading {self.model_name} on {self.device} with {quantization} quantization...")

        # 🧩 Configure quantization (if enabled)
        quant_config = None
        if quantization in ["8bit", "4bit"]:
            try:
                quant_config = BitsAndBytesConfig(
                    load_in_8bit=(quantization == "8bit"),
                    load_in_4bit=(quantization == "4bit"),
                    bnb_4bit_quant_type="nf4",
                    bnb_4bit_use_double_quant=True,
                    bnb_4bit_compute_dtype=dtype,
                )
                print(f"⚙️ Using {quantization} quantization with bitsandbytes.")
            except Exception as e:
                print("⚠️ Quantization not available, loading normally:", e)
                quant_config = None

        # 🧠 Load processor and model
        self.processor = AutoProcessor.from_pretrained(self.model_name)
        self.model = AutoModelForImageTextToText.from_pretrained(
            self.model_name,
            torch_dtype=dtype,
            device_map="auto" if quant_config else None,
            quantization_config=quant_config,
        ).to(self.device if not quant_config else None)

        print(f"✅ {model_size.upper()} SmolVLM2 model loaded successfully!")

    # -------------------------------------------------------------------------
    def _auto_select_model_size(self):
        """Automatically select model size based on available GPU VRAM."""
        if not torch.cuda.is_available():
            print("⚠️ CUDA not available. Using CPU-friendly 256M model.")
            return "small"

        try:
            total_vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
            print(f"🧮 Detected VRAM: {total_vram_gb:.1f} GB")

            if total_vram_gb < 6:
                return "small"
            elif total_vram_gb < 10:
                return "medium"
            else:
                return "large"
        except Exception as e:
            print("⚠️ Could not detect VRAM:", e)
            return "medium"

    # -------------------------------------------------------------------------
    def _format_messages(self, messages):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list of message dicts.")
        return messages

    # -------------------------------------------------------------------------
    def _generate(self, formatted_messages, max_new_tokens=512, do_sample=False):
        inputs = self.processor.apply_chat_template(
            formatted_messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt",
        ).to(self.device, dtype=torch.bfloat16)

        outputs = self.model.generate(
            **inputs,
            do_sample=do_sample,
            max_new_tokens=max_new_tokens,
        )

        return self.processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()

    # -------------------------------------------------------------------------
    def invoke(self, input):
        formatted_messages = self._format_messages([input])
        return {"type": "ai_message", "content": self._generate(formatted_messages)}

    # -------------------------------------------------------------------------
    def batch(self, inputs):
        results = []
        for inp in inputs:
            formatted_messages = self._format_messages([inp])
            response = self._generate(formatted_messages)
            results.append({"type": "ai_message", "content": response})
        return results

    # -------------------------------------------------------------------------
    def trim_video(self, video_path: str, clip_duration: int = 20):
        """
        Trim a long video into multiple 20s clips using the new MoviePy API
        and store them under `trimmed_videos/` beside the original video.
        """
        video_path = Path(video_path)
        output_dir = video_path.parent / "trimmed_videos"
        output_dir.mkdir(exist_ok=True)

        print(f"✂️ Trimming video '{video_path.name}' into {clip_duration}s clips...")

        clips = []
        with VideoFileClip(str(video_path)) as video:
            total_duration = int(video.duration)

            for i, start_time in enumerate(range(0, total_duration, clip_duration)):
                end_time = min(start_time + clip_duration, total_duration)

                # Trim and reduce volume (no text overlay)
                trimmed = video.subclipped(start_time, end_time).with_volume_scaled(0.8)
                output_path = output_dir / f"{video_path.stem}_part{i+1}.mp4"

                trimmed.write_videofile(
                    str(output_path),
                    codec="libx264",
                    audio_codec="aac",
                )
                clips.append(output_path)
                print(f"✅ Saved: {output_path.name}")

        return clips

    # -------------------------------------------------------------------------
    def analyze_video(self, video_path: str, user_query: str = "Describe the main actions, objects, and sequence of events."):
        """
        Trim a video into 20-second clips and analyze each sequentially.
        """
        clips = self.trim_video(video_path, clip_duration=20)
        results = []

        print(f"🔎 Analyzing {len(clips)} clips...")

        for idx, clip_path in enumerate(clips, start=1):
            print(clip_path)
            print(f"\n🎥 Analyzing Clip {idx}/{len(clips)}: {clip_path.name}")

            message = {
                    "role": "user",
                    "content": [
                        {"type": "video", "path": video_path},
                        {"type": "text", "text": user_query}
                    ]
            }

            result = self.invoke(message)
            results.append({"clip": clip_path.name, "analysis": result["content"]})

        return results
    


In [20]:
model = SmolVLM2ChatModel(model_size="medium", quantization="4bit")



🔹 Loading HuggingFaceTB/SmolVLM2-500M-Instruct on cuda with 4bit quantization...
⚙️ Using 4bit quantization with bitsandbytes.
✅ MEDIUM SmolVLM2 model loaded successfully!


In [22]:
video_path = r"C:\Interview Assignment\Intel GenAI Software Engineer Assessment\Short-Video-Analyst\sample_videos\What is MCP and how does it change AI_ (MCP explained) #ai #artificialintelligence.mp4"

video_path = r"C:\Interview Assignment\Intel GenAI Software Engineer Assessment\Short-Video-Analyst\sample_videos\trimmed_videos\What is MCP and how does it change AI_ (MCP explained) #ai #artificialintelligence_part1.mp4"
#video_path = r"C:\Interview Assignment\Intel GenAI Software Engineer Assessment\Short-Video-Analyst\sample_videos\cat.mp4"
user_query = "What did you see in the video?"

results = model.analyze_video(video_path, user_query=user_query)

for r in results:
    print(f"\n🎬 {r['clip']} → {r['analysis']}")

✂️ Trimming video 'What is MCP and how does it change AI_ (MCP explained) #ai #artificialintelligence_part1.mp4' into 20s clips...


MemoryError: Unable to allocate 3.05 MiB for an array with shape (400000,) and data type float64

In [4]:
results

[{'clip': 'cat_part1.mp4',
  'analysis': 'User: Describe the main actions, objects, and sequence of events.<video>\nAssistant: The video shows a person standing in front of a large, modern building with a glass facade. The person is wearing a black jacket and a black hat, and they are holding a black bag. The building has a large glass window on the side, and the person appears to be looking out at the view.\n\nThe video then cuts to a close-up of a person sitting at a desk in a room with a large window. The person is wearing a black shirt and a black hat, and they are holding a black bag. The room has a large window on the side, and the person appears to be looking out at the view.\n\nThe video then cuts to a close-up of a person sitting at a desk in a room with a large window. The person is wearing a black shirt and a black hat, and they are holding a black bag. The room has a large window on the side, and the person appears to be looking out at the view.\n\nThe video then cuts to a 